In [1]:
from ortools.sat.python import cp_model

#initialize every variable constraint

names = ["Carol", "Elisa", "Oliver","Lucas"]

universities = ["Cambridge", "Oxford", "Edinburgh", "London"]

origin_country=["USA","SA","Canada","Australia"]

courses= ["history","medicine","law","architecture"]

genders=["boy","girl"]

In [2]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, universities, origin_country, courses,genders):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.origin_country = origin_country
        self.universities = universities
        self.courses = courses
        self.genders=genders
        self.solutions_ = 0

    def OnSolutionCallback(self):
        self.solutions_ = self.solutions_ + 1
        print("solution", self.solutions_ )
        
        for name in names:
            print(" - "+name+":")
            for country in origin_country:
                if (self.Value(self.origin_country[name][country])):
                    print("origin country : ", country)
            for college in universities:
                if (self.Value(self.universities[name][college])):
                    print("college going : ", college)
            for course in courses:
                if (self.Value(self.courses[name][course])):
                    print("course studying: ", course)
            for gender in genders:
                if (self.Value(self.genders[name][gender])):
                    print("gender of the individual: ", gender)
        
        print()

In [3]:
model = cp_model.CpModel()

#make a dictionary of all the possible countries that my name can take
name_origin_country = {}
for name in names:        
    variables = {}
    for country in origin_country:    
        variables[country] = model.NewBoolVar(name+country)
    name_origin_country[name] = variables

name_university = {}
for name in names:        
    variables = {}
    for college in universities:    
        variables[college] = model.NewBoolVar(name+college)
    name_university[name] = variables

name_course = {}
for name in names:        
    variables = {}
    for course in courses:    
        variables[course] = model.NewBoolVar(name+course)
    name_course[name] = variables
    
name_gender = {}
for name in names:        
    variables = {}
    for gender in genders:    
        variables[gender] = model.NewBoolVar(name+gender)
    name_gender[name] = variables

In [4]:
print(name_origin_country)
print("\n")
print(name_course)
print("\n")
print(name_university)

{'Carol': {'USA': CarolUSA(0..1), 'SA': CarolSA(0..1), 'Canada': CarolCanada(0..1), 'Australia': CarolAustralia(0..1)}, 'Elisa': {'USA': ElisaUSA(0..1), 'SA': ElisaSA(0..1), 'Canada': ElisaCanada(0..1), 'Australia': ElisaAustralia(0..1)}, 'Oliver': {'USA': OliverUSA(0..1), 'SA': OliverSA(0..1), 'Canada': OliverCanada(0..1), 'Australia': OliverAustralia(0..1)}, 'Lucas': {'USA': LucasUSA(0..1), 'SA': LucasSA(0..1), 'Canada': LucasCanada(0..1), 'Australia': LucasAustralia(0..1)}}


{'Carol': {'history': Carolhistory(0..1), 'medicine': Carolmedicine(0..1), 'law': Carollaw(0..1), 'architecture': Carolarchitecture(0..1)}, 'Elisa': {'history': Elisahistory(0..1), 'medicine': Elisamedicine(0..1), 'law': Elisalaw(0..1), 'architecture': Elisaarchitecture(0..1)}, 'Oliver': {'history': Oliverhistory(0..1), 'medicine': Olivermedicine(0..1), 'law': Oliverlaw(0..1), 'architecture': Oliverarchitecture(0..1)}, 'Lucas': {'history': Lucashistory(0..1), 'medicine': Lucasmedicine(0..1), 'law': Lucaslaw(0..

In [5]:
# every name has a different entity
for i in range(4): #iterate through all the names
    for j in range(i+1,4): #iterate through all the names afterwards
        for k in range(4): #iterate through all the countries , universities , courses
            model.AddBoolOr([
                    name_origin_country[names[i]][origin_country[k]].Not(), 
                    name_origin_country[names[j]][origin_country[k]].Not()])
            model.AddBoolOr([
                name_university[names[i]][universities[k]].Not(),
                             name_university[names[j]][universities[k]].Not()])
            model.AddBoolOr([
                name_course[names[i]][courses[k]].Not(), 
                name_course[names[j]][courses[k]].Not()])


model.AddBoolAnd([name_gender["Carol"]["girl"]])
model.AddBoolAnd([name_gender["Elisa"]["girl"]])
model.AddBoolAnd([name_gender["Oliver"]["boy"]])
model.AddBoolAnd([name_gender["Lucas"]["boy"]])


for name in names:
    # at least one entity per name
    variables = []
    for country in origin_country:
        variables.append(name_origin_country[name][country])
    model.AddBoolOr(variables)

    variables = []
    for college in universities:
        variables.append(name_university[name][college])
    model.AddBoolOr(variables)

    variables = []
    for course in courses:
        variables.append(name_course[name][course])
    model.AddBoolOr(variables)

    # max one entity per name
    for i in range(4):
        for j in range(i+1,4):
            model.AddBoolOr([
                    name_origin_country[name][origin_country[i]].Not(), 
                    name_origin_country[name][origin_country[j]].Not()])
            model.AddBoolOr([
                    name_university[name][universities[i]].Not(), 
                    name_university[name][universities[j]].Not()])
            model.AddBoolOr([
                    name_course[name][courses[i]].Not(), 
                    name_course[name][courses[j]].Not()])
            
    #to restrict only one gender
    for i in range(2):
        for j in range(i+1,2):
            model.AddBoolOr([
                    name_gender[name][genders[i]].Not(), 
                    name_gender[name][genders[j]].Not()])


In [6]:
#One of them is going to London (1).

#this is already contained in the first few assumptions that we made



In [7]:
#Exactly one boy and one girl chose a university in a city with the same initial of their names (2).

#carol is going to cambridge if elisa is not going to edinburgh
model.AddBoolOr([name_university[names[0]][universities[0]]]).OnlyEnforceIf([name_university[names[1]][universities[2]].Not()])
#and oliver is going to Oxford if lucas is not going to london
model.AddBoolAnd([name_university[names[2]][universities[1]]]).OnlyEnforceIf([name_university[names[3]][universities[3]].Not()])
#carol is not going to cambridge if elisa is going to edinburgh
model.AddBoolOr([name_university[names[0]][universities[0]].Not()]).OnlyEnforceIf([name_university[names[1]][universities[2]]])
# and oliver is not going to oxford if lucas is going to london
model.AddBoolAnd([name_university[names[2]][universities[1]].Not()]).OnlyEnforceIf([name_university[names[3]][universities[3]]])

In [8]:
#A boy is from Australia, the other studies History (3).

#lucas is from austrailia if oliver studies history
model.AddBoolOr([name_origin_country[names[3]][origin_country[3]]]).OnlyEnforceIf([name_course[names[2]][courses[0]]])
#oliver is from austrailia if lucas studies history
model.AddBoolOr([name_origin_country[names[2]][origin_country[3]]]).OnlyEnforceIf([name_course[names[3]][courses[0]]])
#Elisa can't be from austraila or study history, carol can't be from austrailia or study history
model.AddBoolAnd([name_origin_country[names[1]][origin_country[3]].Not(),name_course[names[0]][courses[0]].Not(),
                 name_origin_country[names[0]][origin_country[3]].Not(),name_course[names[1]][courses[0]].Not()])


In [9]:
#A girl goes to Cambridge, the other studies Medicine (4).

#carol goes to cambridge if elisa studies medicine
model.AddBoolOr([name_university[names[0]][universities[0]]]).OnlyEnforceIf([name_course[names[1]][courses[1]]])
#elisa goes to cambridge if carol studies medicine
model.AddBoolOr([name_university[names[1]][universities[0]]]).OnlyEnforceIf([name_course[names[0]][courses[1]]])
#oliver cant go to cambridge and he can't study medicine 
#lucas cant go to cambridge and he can't study medicine
model.AddBoolAnd([name_university[names[2]][universities[0]].Not(),name_course[names[2]][courses[1]].Not(),
                  name_university[names[3]][universities[0]].Not(),name_course[names[3]][courses[1]].Not()])

In [10]:
#Oliver studies Law or is from USA; He is not from South Africa (5).
#oliver studies Law if he is not from usa
model.AddBoolOr([name_course[names[2]][courses[2]]]).OnlyEnforceIf([name_origin_country[names[2]][origin_country[0]].Not()])
#oliver does not study law if he is from US
model.AddBoolOr([name_course[names[2]][courses[2]].Not()]).OnlyEnforceIf([name_origin_country[names[2]][origin_country[0]]])
#oliver is not from south africa for sure
model.AddBoolAnd([name_origin_country[names[2]][origin_country[1]].Not()])

In [11]:
#The student from Canada is a historian or will go to Oxford (6)
# a student in the list of student
for name in names:
    #this student can study history or go to oxford if he is from Canada
    model.AddBoolOr([name_course[name][courses[0]],name_university[name][universities[1]]]).OnlyEnforceIf([name_origin_country[name][origin_country[2]]])

In [12]:
#The student from South Africa is going to Edinburgh or will study Law (7).
for name in names:
    model.AddBoolOr([name_course[name][courses[2]],name_university[name][universities[2]]]).OnlyEnforceIf([name_origin_country[name][origin_country[1]]])

In [13]:
solver = cp_model.CpSolver()    
solver.SearchForAllSolutions(model, SolutionPrinter(name_university, name_origin_country, name_course,name_gender))

solution 1
 - Carol:
origin country :  SA
college going :  Cambridge
course studying:  law
gender of the individual:  girl
 - Elisa:
origin country :  Canada
college going :  Oxford
course studying:  medicine
gender of the individual:  girl
 - Oliver:
origin country :  USA
college going :  Edinburgh
course studying:  history
gender of the individual:  boy
 - Lucas:
origin country :  Australia
college going :  London
course studying:  architecture
gender of the individual:  boy



4

In [14]:
#decision analytics task 2
#reading the data
import pandas as pd

file_name="Assignment_DA_1_data.xlsx"

xls = pd.ExcelFile(file_name)

df1 = pd.read_excel(xls, 'Projects')
df2 = pd.read_excel(xls, 'Quotes')
df3 = pd.read_excel(xls, 'Dependencies')
df4 = pd.read_excel(xls, 'Value')

df1=df1.fillna(0)
df2=df2.fillna(0)
df3=df3.fillna(0)
df4=df4.fillna(0)


shape_df1=df1.shape
shape_df2=df2.shape
shape_df3=df3.shape
shape_df4=df4.shape

total_projects=shape_df1[0]
total_months=shape_df1[1]
total_contractors=shape_df2[0]
total_jobs=shape_df2[1]

all_projects = range(total_projects)
all_months = range(1,total_months)
all_contractors = range(total_contractors)
all_jobs = range(1,total_jobs)

#get unique job names where the 
job_names={}
for i in all_projects:
    for j in all_months:
        if(df1.iloc[i,j]!=0 ):
            job_names[(i,j)]=df1.iloc[i,j]

print(job_names)
jobs_to_num={}
unique_jobs=list(set(job_names.values()))
unique_jobs.sort()

for i in range(len(unique_jobs)):
    jobs_to_num[unique_jobs[i]]=i
print(jobs_to_num)


{(0, 1): 'Job A', (0, 2): 'Job B', (0, 3): 'Job C', (1, 3): 'Job G', (1, 4): 'Job K', (1, 5): 'Job M', (2, 4): 'Job H', (2, 5): 'Job E', (2, 6): 'Job G', (2, 7): 'Job B', (2, 8): 'Job E', (3, 2): 'Job D', (3, 3): 'Job F', (3, 4): 'Job I', (3, 5): 'Job H', (4, 8): 'Job J', (4, 9): 'Job A', (5, 9): 'Job B', (5, 10): 'Job C', (5, 11): 'Job K', (6, 5): 'Job L', (6, 6): 'Job M', (6, 7): 'Job E', (7, 8): 'Job A', (7, 9): 'Job B', (7, 10): 'Job D', (7, 11): 'Job I', (8, 10): 'Job L', (8, 11): 'Job F', (8, 12): 'Job K'}
{'Job A': 0, 'Job B': 1, 'Job C': 2, 'Job D': 3, 'Job E': 4, 'Job F': 5, 'Job G': 6, 'Job H': 7, 'Job I': 8, 'Job J': 9, 'Job K': 10, 'Job L': 11, 'Job M': 12}


In [18]:
model = cp_model.CpModel()

def get_all_contractors(job_name):
    """takes the name of the job and returns all the possible contractors available"""
    contractors=list(df2[job_name])
    avl_contractors=[]
    for i,element in enumerate(contractors):
        if(element !=0):
            avl_contractors.append(i)
    
    return avl_contractors
    
avl_contractors=get_all_contractors("Job A")    
    

#Assign sheet 1
projects_months_contractors = {} #contractors available
for p in all_projects:
    for m in all_months:
            #select possible job blocks
            if((p,m) in job_names ):
                #assign only those jobs that are possible
                get_job=job_names[(p,m)] #this is a text
                j=jobs_to_num[get_job] #this is a number 
                #get contractors list possible for this job
                allpossible_contractors=get_all_contractors(get_job)
                for c in allpossible_contractors:
                    projects_months_contractors[(p,m,c)]= model.NewBoolVar('project%i_month%i_c%i' % (p, m,c))
                    
# equals 1 if job j is assigned to contractor c on month m, and 0 if the task is not assigned
print(projects_months_contractors)


#exactly one contractor works on the a particular job 

#each job has to be done by a contractor which can select or not select
for p in all_projects:
    for m in all_months:
        if((p,m) in job_names): #if there exists a job
            get_job=job_names[(p,m)] #this is a text
            j=jobs_to_num[get_job] #this is a number 
            allpossible_contractors=get_all_contractors(get_job)
            model.Add(sum(projects_months_contractors[(p,m,c)] for c in allpossible_contractors ) <= 1)

#no contractor can work on two jobs at the same time 
#this means if project is different month we will get some job values for all the contractors willing to work on this problem 
#they can be maximum 1

#for all the months all projects get the jobs available and see for all the occupied spaces 
#if the space is occupied then we can 

for p in all_projects: #traverse through all projects
    for m in all_months: #traverse through all months 
        temp_list=[] #make an empty list which will be later used to assign conditions  
        for element in projects_months_contractors: #check if the variable already exist in the (p,m,c) variables
            if (element[0] == p and element[1] == m): #check for the first two elements 
                #not all the values will be selected as the index wont match for the blank spots
                temp_list.append(projects_months_contractors[element]) #if the two match with the same value then
                #we can add this value to the constraint 
        model.Add(sum(temp_list)==1)



{(0, 1, 0): project0_month1_c0(0..1), (0, 1, 7): project0_month1_c7(0..1), (0, 2, 4): project0_month2_c4(0..1), (0, 2, 9): project0_month2_c9(0..1), (0, 3, 5): project0_month3_c5(0..1), (0, 3, 10): project0_month3_c10(0..1), (1, 3, 2): project1_month3_c2(0..1), (1, 3, 6): project1_month3_c6(0..1), (1, 3, 8): project1_month3_c8(0..1), (1, 4, 1): project1_month4_c1(0..1), (1, 4, 10): project1_month4_c10(0..1), (1, 5, 1): project1_month5_c1(0..1), (1, 5, 10): project1_month5_c10(0..1), (2, 4, 7): project2_month4_c7(0..1), (2, 4, 10): project2_month4_c10(0..1), (2, 5, 0): project2_month5_c0(0..1), (2, 5, 4): project2_month5_c4(0..1), (2, 6, 2): project2_month6_c2(0..1), (2, 6, 6): project2_month6_c6(0..1), (2, 6, 8): project2_month6_c8(0..1), (2, 7, 4): project2_month7_c4(0..1), (2, 7, 9): project2_month7_c9(0..1), (2, 8, 0): project2_month8_c0(0..1), (2, 8, 4): project2_month8_c4(0..1), (3, 2, 3): project3_month2_c3(0..1), (3, 2, 7): project3_month2_c7(0..1), (3, 3, 5): project3_month3_c5

In [39]:
#project cost with budget

for p in all_projects:
    total_price=[]
    for m in all_months:
        if((p,m) in job_names): #if there exists a job
            get_job=job_names[(p,m)] #this is a text
            j=jobs_to_num[get_job] #this is a number 
            allpossible_contractors=get_all_contractors(get_job)
            #get the price for all the possible contractors
            current=[]
            for c in allpossible_contractors:
                price=df2.iloc[c,j+1]
                current.append((price,c))
            #select the minimum of the possible prices for
            #print(current)
            selected=min(current)
            total_price.append(selected)
    fprice=0
    contractor_names=[]
    for element in total_price:
        fprice+=element[0]
        contractor_names.append(element[1])
    print(fprice,contractor_names)
    if(fprice<df4.iloc[p,1]):
        print("valid solution with savings: {}".format(df4.iloc[p,1]-fprice))
    else:
        print("not valid solution at all")

510.0 [0, 4, 10]
not valid solution at all
935.0 [6, 1, 1]
not valid solution at all
795.0 [7, 4, 6, 4, 4]
not valid solution at all
720.0 [7, 5, 6, 7]
valid solution with savings: 280.0
420.0 [2, 0]
valid solution with savings: 1580.0
420.0 [4, 10, 1]
not valid solution at all
500.0 [3, 1, 4]
valid solution with savings: 1000.0
820.0 [0, 4, 7, 6]
valid solution with savings: 180.0
470.0 [3, 5, 1]
valid solution with savings: 530.0


class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, project, projects_months_contractors):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.project_ = project
        self.projects_months_contractors = projects_months_contractors
        self.solutions_ = 0

    def OnSolutionCallback(self):
        self.solutions_ = self.solutions_ + 1
        print("solution", self.solutions_)
        for project in projects:
            print(" - " + project + ":")
            for projects_months_contractors in pr_con_mon_job:
                if (self.Value(self.projects_months_contractors[project][contractor_month_job])):
                    print(contractor, project, month, job)
        print()

solver = cp_model.CpSolver()
solver.SearchForAllSolutions(model, SolutionPrinter(all_projects, projects_months_contractors))